In [9]:
library(tidyverse)
library(biomaRt)
library(OmnipathR)
library(annotatepip)

## Retrieve biomRt database

In [10]:
options(timeout = 600)

In [12]:
import_biomart_local()
ls()

Warning message in file(file, "rt"):
"cannot open file '/Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/library/annotatepip/extdatarelease-113.tsv': No such file or directory"


ERROR: Error in file(file, "rt"): cannot open the connection


In [47]:
biomart_dict %>% head(.)

,mouse_gene_symbol,mouse_chromosome,human_gene_symbol,human_chromosome
,<chr>,<chr>,<chr>,<chr>
1,mt-Nd1,MT,MT-ND1,MT
2,mt-Nd4,MT,MT-ND4,MT
3,mt-Nd2,MT,MT-ND2,MT
4,mt-Nd5,MT,MT-ND5,MT
5,mt-Nd6,MT,MT-ND6,MT
6,mt-Atp6,MT,MT-ATP6,MT


In [33]:
setwd("/Users/hungm/Documents/development/annotatepip/")
write.table(biomart_dict, "inst/extdata/release-105.tsv", col.names = T, row.names = F, quote = F, sep = "\t")

## Build omnipath database

In [2]:
omnipathdb <- import_omnipath_intercell()

In [3]:
##
omnipathdb <- omnipathdb %>%
    mutate(
        human_gene_symbol =  genesymbol,
        database = ifelse(str_detect(database, "UniProt"), "UniProt", database)) %>%
    merge(biomart_dict, ., by = "human_gene_symbol", all = F) %>%
    dplyr::select(-c("genesymbol"))
omnipathdb %>% head(.)

ERROR: Error: object 'biomart_dict' not found


In [ ]:
omnipathdb <- omnipathdb %>%
    dplyr::select(human_gene_symbol, mouse_gene_symbol, entity_type, category, parent, consensus_score, database) %>%
    arrange(human_gene_symbol) %>%
    group_by(human_gene_symbol) %>%
    summarize(
        mouse_gene_symbol = unique(mouse_gene_symbol),
        entity_type = unique(entity_type),
        category = paste0(unique(category), collapse = ", "),
        parent = paste0(unique(parent), collapse = ", "),
        consensus_score = max(consensus_score, na.rm = T),
        database = paste0(unique(database), collapse = ", ")) %>%
    ungroup()

omnipathdb %>% head(.)

Warning message:
"Returning more (or less) than 1 row per `summarise()` group was deprecated in
dplyr 1.1.0.
i Please use `reframe()` instead.
i When switching from `summarise()` to `reframe()`, remember that `reframe()`
  always returns an ungrouped data frame and adjust accordingly."
`summarise()` has grouped output by 'human_gene_symbol'. You can override using
the `.groups` argument.


human_gene_symbol,mouse_gene_symbol,entity_type,category,parent,consensus_score,database
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
A1BG,A1bg,protein,"extracellular, secreted, intracellular, receptor","extracellular, secreted, intracellular, receptor",3,"LOCATE, UniProt, ComPPI, OmniPath, CellTalkDB, GO_Intercell, HPA_secretome"
A1CF,A1cf,protein,intracellular,intracellular,3,"OmniPath, LOCATE, GO_Intercell, UniProt, ComPPI"
A2M,A2m,protein,"extracellular, intracellular, secreted, ligand_regulator, growth_factor_binder, ecm_regulator","extracellular, intracellular, secreted, ligand_regulator, ecm_regulator",5,"OmniPath, LOCATE, HPA_secretome, connectomeDB2020, Baccin2019, UniProt, GO_Intercell, Matrisome"
A3GALT2,A3galt2,protein,"transmembrane, intracellular","transmembrane, intracellular, transmembrane_predicted",3,"UniProt, OmniPath, GO_Intercell, Phobius"
A4GALT,A4galt,protein,"transmembrane, intracellular, transmembrane_tmhmm, transmembrane_sosui, transmembrane_phobius, transmembrane_predicted","transmembrane, intracellular, transmembrane_predicted",6,"UniProt, GO_Intercell, ComPPI, Almen2009, Ramilowski_location, OmniPath, LOCATE, Phobius"
A4GNT,A4gnt,protein,"transmembrane, intracellular","transmembrane, intracellular",5,"LOCATE, UniProt, OmniPath, Ramilowski_location, GO_Intercell"


In [37]:
dorotheadb <- dorothea(organism = 9606)
dorotheadb %>% head(.)

source,target,source_genesymbol,target_genesymbol,is_directed,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,sources,references,curation_effort,dorothea_level,n_references,n_resources
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>
P01106,O14746,MYC,TERT,1,1,0,1,1,0,DoRothEA;HTRIdb_DoRothEA;KEGG_DoRothEA;RegNetwork_DoRothEA;TFactS_DoRothEA;TRED_DoRothEA;TRRUST_DoRothEA,,7,A,0,1
P84022,P05412,SMAD3,JUN,1,1,0,1,1,0,DoRothEA;RegNetwork_DoRothEA;TFactS_DoRothEA;TRED_DoRothEA;TRRUST_DoRothEA,DoRothEA:10022869,6,A,1,1
Q13485,P05412,SMAD4,JUN,1,1,0,1,1,0,DoRothEA;RegNetwork_DoRothEA;TFactS_DoRothEA;TRED_DoRothEA;TRRUST_DoRothEA,DoRothEA:10022869,6,A,1,1
P08047,P04075,SP1,ALDOA,1,1,0,1,1,0,DoRothEA;ORegAnno_DoRothEA;RegNetwork_DoRothEA;TFactS_DoRothEA;TRED_DoRothEA;TRRD_DoRothEA,DoRothEA:8524331,7,A,1,1
P04637,P08069,TP53,IGF1R,1,0,1,1,0,1,DoRothEA;PAZAR_DoRothEA;ReMap_DoRothEA;RegNetwork_DoRothEA;TRED_DoRothEA;TRRUST_DoRothEA,,6,A,0,1
Q05516,P20248,ZBTB16,CCNA2,1,0,1,1,0,1,DoRothEA;KEGG_DoRothEA;TRRUST_DoRothEA,DoRothEA:20731660,4,A,1,1


In [38]:
omnipathdb$dorothea_tf <- ifelse(omnipathdb$human_gene_symbol %in% dorotheadb$source_genesymbol, T, F)
omnipathdb$intercell_cs <- ifelse(str_detect(omnipathdb$parent, "transmembrane|cell_surface|adhesion|junction"), T, F)

In [39]:
write.table(omnipathdb, "inst/extdata/release-105_omnipathdb.tsv", col.names = T, row.names = F, quote = F, sep = "\t")

In [40]:
sessionInfo()

R version 4.4.0 (2024-04-24)
Platform: aarch64-apple-darwin20
Running under: macOS Sonoma 14.6.1

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/lib/libRblas.0.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/lib/libRlapack.dylib;  LAPACK version 3.12.0

locale:
[1] C

time zone: Europe/London
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] OmnipathR_3.12.4 biomaRt_2.60.1   lubridate_1.9.4  forcats_1.0.0   
 [5] stringr_1.5.1    dplyr_1.1.4      purrr_1.0.4      readr_2.1.5     
 [9] tidyr_1.3.1      tibble_3.2.1     ggplot2_3.5.1    tidyverse_2.0.0 

loaded via a namespace (and not attached):
 [1] tidyselect_1.2.1        IRdisplay_1.1           blob_1.2.4             
 [4] filelock_1.0.3          Biostrings_2.72.1       fastmap_1.2.0          
 [7] BiocFileCache_2.12.0    digest_0.6.37           timechange_0